# Exploring Board Game Geek 

[BoardGameGeek](https://boardgamegeek.com/) (BGG) is a game database with over 125,600 different tabletop games, including European-style board games, wargames, and card games. In addition to the game database, the site allows users to rate games on a 1–10 scale and publishes a ranked list of board games. 

The dataset being used for this project is from [kaggle](https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek), sourced from the BGG API. 

# Imports

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats

import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
boardgames_df = pd.read_csv('data/games.csv')

In [ ]:
users_df = pd.read_csv('data/user_ratings.csv')

In [597]:
game_mechanics_df = pd.read_csv('data/mechanics.csv')
game_themes_df = pd.read_csv('data/themes.csv')

In [598]:
#define a common color pallete for all graphs
color_pallete = px.colors.qualitative.Pastel
color_pallete_continuous = color_pallete[0:2]

# Game Overviews 

In [599]:
boardgames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int64  
 1   Name                 21925 non-null  object 
 2   Description          21924 non-null  object 
 3   YearPublished        21925 non-null  int64  
 4   GameWeight           21925 non-null  float64
 5   AvgRating            21925 non-null  float64
 6   BayesAvgRating       21925 non-null  float64
 7   StdDev               21925 non-null  float64
 8   MinPlayers           21925 non-null  int64  
 9   MaxPlayers           21925 non-null  int64  
 10  ComAgeRec            16395 non-null  float64
 11  LanguageEase         16034 non-null  float64
 12  BestPlayers          21925 non-null  int64  
 13  GoodPlayers          21925 non-null  object 
 14  NumOwned             21925 non-null  int64  
 15  NumWant              21925 non-null 

In [629]:
boardgames_df.head()

0    https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...
2    https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...
3    https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...
4    https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...
5    https://cf.geekdo-images.com/277POF80AUz2ZE9XS...
Name: ImagePath, dtype: object

The categories and rankings are not very accurate, so we will drop those and add our own later. 
We will also drop games that are reimplementations of older games. 

In [601]:
boardgames_df = boardgames_df.drop(columns=['Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts', 'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames', 'Rank:partygames', 'Rank:childrensgames',
                                            'Cat:Thematic', 'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens'])
boardgames_df = boardgames_df.loc[boardgames_df['IsReimplementation'] == 0]
boardgames_df = boardgames_df.drop(columns=['NumImplementations', 'IsReimplementation', 'NumAlternates', 'NumExpansions', 'NumComments', 'LanguageEase', 'NumWish'])
boardgames_df.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Family,Kickstarted,ImagePath
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,501,761,240,240,240,14,5354,Classic Line (Valley Games),0,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,799,1451,60,30,60,10,15146,Euro Classics (Reiner Knizia),0,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,54,30,60,60,60,12,340,NaN,0,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,548,1606,90,90,90,12,18655,3M Bookshelf,0,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...
5,6,Mare Mediterraneum,ancient land mediterranean player attempt sati...,1989,3.0000,6.55370,5.54614,1.65350,2,6,...,34,7,240,240,240,12,81,NaN,0,https://cf.geekdo-images.com/277POF80AUz2ZE9XS...


There are a lot of games! However, some are VERY old.

In [602]:
fig = px.histogram(boardgames_df, x= 'YearPublished', color_discrete_sequence = color_pallete)
fig.show()

While it's very cool to look at how long humans have been making board games (and how someone has mislabled 'Dog-opoly' as having been published in 0BC) we are looking to show users more modern games. 

In [603]:
modern_boardgames_df = boardgames_df.loc[boardgames_df['YearPublished']>=1960]

In [604]:
fig = px.histogram(modern_boardgames_df, x= 'YearPublished', color_discrete_sequence = color_pallete)

fig.add_annotation(x=1995, y=252,
            text="Settlers of Catan Released",
            showarrow=True,
            arrowhead=1)

fig.update_layout(
    xaxis_title_text='Year', # xaxis label
    yaxis_title_text='Number of Games Published', # yaxis label
)

fig.show()

That is easier to look at! It is often said that the popularity of 'Settlers Of Catan' led to a board game explosion, and we certainly see more games published ech year afterwards. 

## What are the families? 

In [605]:
modern_boardgames_df['Family'].value_counts(sort=True)[:25]

Family
18xx                                         94
Trivial Pursuit                              62
Monopoly (Official)                          61
Werewolf / Mafi                              46
Wallet Games (Button Shy)                    39
UNO                                          38
Two-player games (Kosmos)                    37
LEGO Gam                                     34
Monopoly-Lik                                 31
Cube Rail                                    29
Cluedo / Clue (Parker Brothers, et al)       29
Unlock! (Space Cowboys)                      29
Exit – The Gam                               28
Anno Domini                                  28
Black Stories                                27
Fantasy Flight Silver Lin                    27
Jours de Gloire (Frédéric Bey)               26
Rory's Story Cubes                           26
Scene It?                                    26
MicroGame (Metagaming)                       25
Mitbringspiele (HABA)            

The families are company related - for example Wallet Games (Button Shy) refers to a publishing company based in Cherry Hill, NJ that make little games that fit in little card sleeves. Monopoly is simply.... monopoly. 

However, we don't really need this as a feature, as so few games are tagged this way. 

In [606]:
modern_boardgames_df = modern_boardgames_df.drop(columns=['Family'])

## How Many Ratings Does Each Game Have?


In [607]:
fig = px.histogram(modern_boardgames_df, y= 'NumUserRatings', color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Number of Games', # xaxis label
    yaxis_title_text='Number of Reviews', # yaxis label
)

fig.show()

In [608]:
sort = modern_boardgames_df.sort_values(by= 'NumUserRatings',ascending=False)[:5]
sort

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,NumOwned,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Kickstarted,ImagePath
7919,30549,Pandemic,pandemic virulent disease break simultaneously...,2008,2.4072,7.59130,7.48919,1.32867,2,4,...,166497,630,5585,45,45,45,8,108101,0,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...
673,822,Carcassonne,carcassonne tileplacement game player draw pla...,2000,1.9064,7.41883,7.30890,1.30503,2,5,...,159709,581,7673,45,30,45,7,107937,0,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...
12,13,Catan,catan settler catan player try dominant force ...,1995,2.3139,7.13746,6.97148,1.48183,3,4,...,165651,484,7508,120,60,120,10,107141,0,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...
9861,68448,7 Wonders,leader great city ancient world gather resou...,2010,2.3258,7.73733,7.63557,1.27611,2,7,...,119235,1003,4757,30,30,30,10,89249,0,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...
8459,36218,Dominion,quotyou monarch like parent ruler small pleasa...,2008,2.3547,7.61081,7.49999,1.42440,2,4,...,106256,652,4988,30,30,30,13,81131,1,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...


In [609]:
sort = modern_boardgames_df.sort_values(by= 'NumUserRatings')[:5]
sort

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,NumOwned,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Kickstarted,ImagePath
3243,5282,Raise the Titanic,object game earn fame fortune explore plunder ...,1987,1.0000,5.11000,5.49682,1.98433,2,4,...,102,5,3,30,30,30,8,30,0,https://cf.geekdo-images.com/P7qNHUk23vHLrSC3d...
6704,20231,Die Pyramide des Krimsutep,game player role archeologist explore pyramid ...,2005,1.0000,4.46667,5.48109,1.11006,2,5,...,85,0,2,90,90,90,10,30,0,https://cf.geekdo-images.com/iSHITH0qUwxMeyWP9...
3626,6221,Caper,player assume role jewel thief attempt steal j...,1970,1.3333,6.31667,5.51069,1.30053,2,4,...,84,16,3,60,60,60,9,30,0,https://cf.geekdo-images.com/hMr39LNJmGNN9eRbE...
15807,194534,Awesome Kingdom: Mines & Labyrinths,description publisherawesome kingdom mine amp ...,2016,3.0000,5.88667,5.50220,1.68992,2,4,...,157,3,1,45,30,45,12,30,0,https://cf.geekdo-images.com/Z8byleUG9Evp8rOAn...
5890,14617,Mikado Magnetico,variant classic mikado pick stick pick stick u...,1988,1.0000,4.80000,5.48899,1.22202,2,4,...,57,0,3,0,0,0,6,30,0,https://cf.geekdo-images.com/ubAY-cZubfH5gOAZZ...


That's a range! The top picks are some of the most popular games on the market, which isn't very suprising. At the lower end of the spectrum, a game needs 30 user ratings to be Ranked on BoardGameGeek, so it is not suprising that that is our minimum. 

For our purposes, we want more popular games, so we will only look at games with over 100 reviews.

In [615]:
modern_boardgames_df = boardgames_df.loc[boardgames_df['NumUserRatings']>=100]
modern_boardgames_df.sort_values(by='NumUserRatings', ascending=False)[:5]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Family,Kickstarted,ImagePath
7919,30549,Pandemic,pandemic virulent disease break simultaneously...,2008,2.4072,7.59130,7.48919,1.32867,2,4,...,630,5585,45,45,45,8,108101,Pandemic,0,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...
673,822,Carcassonne,carcassonne tileplacement game player draw pla...,2000,1.9064,7.41883,7.30890,1.30503,2,5,...,581,7673,45,30,45,7,107937,Carcassonn,0,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...
12,13,Catan,catan settler catan player try dominant force ...,1995,2.3139,7.13746,6.97148,1.48183,3,4,...,484,7508,120,60,120,10,107141,Catan,0,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...
9861,68448,7 Wonders,leader great city ancient world gather resou...,2010,2.3258,7.73733,7.63557,1.27611,2,7,...,1003,4757,30,30,30,10,89249,7 Wonders,0,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...
8459,36218,Dominion,quotyou monarch like parent ruler small pleasa...,2008,2.3547,7.61081,7.49999,1.42440,2,4,...,652,4988,30,30,30,13,81131,Dominion,1,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...


Our most reviewed games are still magnitudes more reviewed than our least reviewed games, but there is substantial data for each game. 

# Average Rating

Board Game Geek Uses a 1-10 Rating system with the following values: 

- 10 - Outstanding. Always want to play and expect this will never change.

- 9 - Excellent game. Always want to play it.

- 8 - Very good game. I like to play. Probably I'll suggest it and will never turn down a game.

- 7 - Good game, usually willing to play.

- 6 - Ok game, some fun or challenge at least, will play sporadically if in the right mood.

- 5 - Average game, slightly boring, take it or leave it.

- 4 - Not so good, it doesn't get me but could be talked into it on occasion.

- 3 - Likely won't play this again although could be convinced. Bad.

- 2 - Extremely annoying game, won't play this ever again.

- 1 - Defies description of a game. You won't catch me dead playing this. Clearly broken.


Only games that have at least 30 User Ratings are eligible to join the site Ranking for top games.


In [617]:
fig = px.histogram(modern_boardgames_df, x= 'AvgRating', color_discrete_sequence = color_pallete)
fig.update_layout(
    xaxis_title_text='Average Rating', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

BGG also provides an 'adjusted' rating based on the number of ratings a board game has overall. You can read more [here](https://boardgamegeek.com/wiki/page/ratings)

In [618]:
fig = px.histogram(modern_boardgames_df, x= 'BayesAvgRating', color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Average Rating Adjusted by Number of Reviews', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

This rating causes games with *few votes* but very high ratings to rank lower than games with *many more votes* but a lower Average Rating. 

It also pushes the scores overall closer to the average rating of all games on the site, around 5.5-6 ("Ok game, some fun or challenge at least, will play sporadically if in the right mood.") using dummy variables : with no way to verify these, I will be using average rating instead. 

In [619]:
modern_boardgames_df = modern_boardgames_df.drop(columns=['BayesAvgRating', 'StdDev'])

# Average Game Weight / Complexity 

Weight is a personal opinion expressing how difficult the game is to play - "Weight" is not actually defined by BGG so different people have different ideas of what it means. The general consensus is that lightweight games are easier to learn, while heavyweight games are more difficult to learn. 

There are only 13 unrated games, so I've made the decision to drop them. 

In [620]:
modern_boardgames_df = modern_boardgames_df.loc[modern_boardgames_df['GameWeight']>0]

In [621]:
fig = px.scatter(modern_boardgames_df, x= 'GameWeight', y = 'AvgRating', color='AvgRating', color_continuous_scale = color_pallete_continuous)

fig.update_layout(
    xaxis_title_text='Average Game Weight', # xaxis label
    yaxis_title_text='Average Rating', # yaxis label
)

fig.show()

There are good games in each game weight, however we can see that the less 'weighty' games have a lower values, while no game above 3 is rated below a 5. 

In [622]:
fig = px.histogram(modern_boardgames_df, x= 'GameWeight', color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Average Game Weight', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

In [623]:
heavyweight = modern_boardgames_df.loc[modern_boardgames_df['GameWeight']>=3]

heavyweight.sort_values('GameWeight', ascending=False)[:3]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,MinPlayers,MaxPlayers,ComAgeRec,BestPlayers,...,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Family,Kickstarted,ImagePath
2686,4102,Europa Universalis,quoteuropa universalisquot monster wargame dip...,1993,4.8537,6.84085,1,6,16.000000,0,...,59,82,3600,3600,3600,14,343,NaN,0,https://cf.geekdo-images.com/Gk4MDKpkH5ekO70SL...
3292,5410,La Grande Guerre 14-18,la grande guerre great war english rule set ...,1999,4.7500,7.80673,2,6,16.000000,0,...,64,24,3600,360,3600,14,104,NaN,0,https://cf.geekdo-images.com/rKL-dNx_ghtZPkz3j...
9607,63170,1817,railroad operation share trading board game ...,2010,4.7162,8.68786,3,7,16.666667,0,...,151,74,540,360,540,16,557,18xx,0,https://cf.geekdo-images.com/ljswtJbDAR_bcgSkF...


In [624]:
lightweight = modern_boardgames_df.loc[modern_boardgames_df['GameWeight']==1]
lightweight.sort_values('GameWeight')[:3]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,MinPlayers,MaxPlayers,ComAgeRec,BestPlayers,...,NumWant,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,Family,Kickstarted,ImagePath
428,499,Arbos,manual dexterity game player attempt place woo...,1999,1.0,6.36683,1,8,6.0,0,...,68,36,30,30,30,6,378,NaN,0,https://cf.geekdo-images.com/sF1IUUa-l6io7EeRq...
16591,208807,Rainbow Rage,game description publisherred orange yellow gr...,2016,1.0,5.81482,2,8,6.0,0,...,4,2,30,30,30,8,114,NaN,0,https://cf.geekdo-images.com/5G6Ji2Il4Etmg91vd...
16549,207911,Trivial Pursuit: 2000s,new edition classic trivia game time test know...,2016,1.0,5.67049,2,6,NaN,0,...,5,4,45,45,45,16,226,Trivial Pursuit,0,https://cf.geekdo-images.com/6BZ7HPTitPX708B_-...


# Multiple Feilds & Correlations

It would be nice to also look at more combined metrics, but as these values are floats, they can be difficult to bin and group. For some more overviews, I've decided to examine average rounded to the nearest whole and 0.5. 

In [625]:
# Function to round to nearest whole number or .5
def round_to_whole_or_pfive(x):
    return round(x * 2) / 2

In [626]:
modern_boardgames_df['Rounded_Rating'] = modern_boardgames_df['AvgRating'].map(round_to_whole_or_pfive)
modern_boardgames_df['Rounded_Weight'] = modern_boardgames_df['GameWeight'].map(round_to_whole_or_pfive)

In [627]:
fig = px.histogram(modern_boardgames_df, x= 'Rounded_Rating', color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Average Rating', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

In [628]:
fig = px.histogram(modern_boardgames_df, x= 'Rounded_Weight', color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Average Game Weight', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

These follow our observations above, so we will use them for some more generalizations. 

Now if we look at the rating by weight:

In [507]:
fig = px.scatter(modern_boardgames_df, x= 'Rounded_Weight', y = 'Rounded_Rating', color='Rounded_Rating', color_continuous_scale = color_pallete_continuous)

fig.update_layout(
    xaxis_title_text='Average Game Weight', # xaxis label
    yaxis_title_text='Average Rating', # yaxis label
)

fig.show()

It's even easier to see that while there are more low-rated row-complexity games, there are good games in every weight class. 

# Adding Mechanics

Among board game nerds, the primary mechanics can be a big deal! For example, I prefer 'engine building' games over 'worker placement'. However... there are 158 categories. 

That is a lot of features!

In [508]:
game_mechanics_df.head()

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,...,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets start by removing games that aren't in our modern game collection.

In [509]:
game_mechanics_df = game_mechanics_df[game_mechanics_df['BGGId'].isin(modern_boardgames_df['BGGId'])]

Then we can examine what these mechanics are.

In [510]:
mechanics_series = pd.Series()
for col in game_mechanics_df.columns:
    if col != 'BGGId':
        mechanics_series[col] = sum(game_mechanics_df[col])

mechanics_series.index

Index(['Alliances', 'Area Majority / Influence', 'Auction/Bidding',
       'Dice Rolling', 'Hand Management', 'Simultaneous Action Selection',
       'Trick-taking', 'Hexagon Grid', 'Once-Per-Game Abilities',
       'Set Collection',
       ...
       'Contracts', 'Passed Action Token', 'King of the Hill',
       'Action Retrieval', 'Force Commitment', 'Rondel',
       'Automatic Resource Growth', 'Legacy Game', 'Dexterity', 'Physical'],
      dtype='object', length=157)

## How common are these different mechanics?

In [511]:
mechanics_series = mechanics_series.sort_values(ascending=False)
fig = px.histogram(mechanics_series, x= mechanics_series.index, y=mechanics_series, color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Mechanic Featured', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

With so many mechanics, it's difficult to see popularity. Lets look at the top picks: 

In [512]:
mechanics_popular = mechanics_series[:35]
fig = px.histogram(mechanics_popular, x= mechanics_popular.index, y=mechanics_popular, color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Mechanic Featured', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

Dice rolling and Hand management are the most common mechanics!  Set colection, Variable Player Powers, and Hexagon grids are also quite popular. 

Now for the least popular:

In [513]:
mechanics_popular = mechanics_series[-35:]
fig = px.histogram(mechanics_popular, x= mechanics_popular.index, y=mechanics_popular, color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Mechanic Featured', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
    yaxis_range=[0,500]
)

fig.show()

As we get into the less common mechanics, we can see that many of these have less than 100 games assosiated with them. 

Therefore, we can drop some of these to reduce our df length. 

In [514]:
droplist = mechanics_popular.index[-100:]

In [515]:
for m in droplist:
    game_mechanics_df = game_mechanics_df.drop(columns=[m])

In [516]:
game_mechanics_df

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,...,"Deck, Bag, and Pool Building",Move Through Deck,Single Loser Game,Narrative Choice / Paragraph,Follow,Contracts,Action Retrieval,Rondel,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21882,342010,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21886,342207,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21896,342942,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21902,343905,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0


In [517]:
mechanics_series = pd.Series()
for col in game_mechanics_df.columns:
    if col != 'BGGId':
        mechanics_series[col] = sum(game_mechanics_df[col])

mechanics_series.index

Index(['Alliances', 'Area Majority / Influence', 'Auction/Bidding',
       'Dice Rolling', 'Hand Management', 'Simultaneous Action Selection',
       'Trick-taking', 'Hexagon Grid', 'Once-Per-Game Abilities',
       'Set Collection',
       ...
       'Deck, Bag, and Pool Building', 'Move Through Deck',
       'Single Loser Game', 'Narrative Choice / Paragraph', 'Follow',
       'Contracts', 'Action Retrieval', 'Rondel', 'Dexterity', 'Physical'],
      dtype='object', length=122)

We can now update our mechanics series. 

## Finding the highest rated mechanics

In [518]:
def mean_mechanic_rating (col):
    temp_group = mechanics_with_ratings_wide.groupby(col).mean()
    return temp_group.iloc[1, temp_group.columns.get_loc('AvgRating')]

In [519]:
mechanics_with_ratings_wide = modern_boardgames_df[['BGGId','AvgRating']]

mechanics_with_ratings_wide = mechanics_with_ratings_wide.merge(game_mechanics_df, on="BGGId")

In [520]:
mechanics_rating = pd.DataFrame()
mechanics_rating['mechanic'] = mechanics_series.index
mechanics_rating['num_games'] = mechanics_series.values

mechanics_rating['AvgRating'] = mechanics_rating['mechanic'].apply(mean_mechanic_rating)
mechanics_rating = mechanics_rating.sort_values(by='AvgRating', ascending=False)
mechanics_rating.head()

,mechanic,num_games,AvgRating
88,Ownership,30,7.644142
94,Hidden Victory Points,22,7.615733
65,Action Drafting,27,7.586261
27,Income,65,7.585058
8,Once-Per-Game Abilities,24,7.556929


In [521]:
temp_mech = mechanics_rating[:15]
fig = px.bar(temp_mech, x='mechanic', y='AvgRating', color_discrete_sequence = color_pallete)
fig.show()

In [522]:
temp_mech = mechanics_rating[-15:]
fig = px.bar(temp_mech, x='mechanic', y='AvgRating', color_discrete_sequence = color_pallete)
fig.show()

# Game Themes

Some of the themes start with 'Theme_' but not all of them

In [523]:
game_themes_df.head()

,BGGId,Adventure,Fantasy,Fighting,Environmental,Medical,Economic,Industry / Manufacturing,Transportation,Science Fiction,...,Theme_Fashion,Theme_Geocaching,Theme_Ecology,Theme_Chernobyl,Theme_Photography,Theme_French Foreign Legion,Theme_Cruise ships,Theme_Apache Tribes,Theme_Rivers,Theme_Flags identification
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [524]:
themes_series = pd.Series()
for col in game_themes_df.columns:
    if col != 'BGGId':
        themes_series[col] = sum(game_themes_df[col])

themes_series

Adventure                      1177
Fantasy                        2702
Fighting                       1668
Environmental                   194
Medical                          87
                               ... 
Theme_French Foreign Legion       2
Theme_Cruise ships                2
Theme_Apache Tribes               2
Theme_Rivers                      2
Theme_Flags identification        2
Length: 217, dtype: int64

In [525]:
themes_series = themes_series.sort_values(ascending=False)
fig = px.histogram(themes_series, x= themes_series.index, y=themes_series, color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Theme', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

In [536]:
themes_popular = themes_series[:25]

fig = px.histogram(themes_popular, x= themes_popular.index, y=themes_popular, color=themes_popular.index, color_discrete_sequence = color_pallete)

fig.update_layout(
    xaxis_title_text='Top Themes', # xaxis label
    yaxis_title_text='Number of Games', # yaxis label
)

fig.show()

In [527]:
#drop themes with less than 50 games. 
themes_series = themes_series.loc[themes_series.values > 50]
themes_series

Fantasy                             2702
Science Fiction                     1675
Fighting                            1668
Economic                            1519
Animals                             1367
                                    ... 
Theme_Archaeology / Paleontology      61
Theme_Witches                         59
Theme_Deserts                         57
Theme_Tropical                        55
Theme_Steampunk                       53
Length: 78, dtype: int64

In [528]:
def re_theme (theme):
    if theme in ["Adventure", "Pirates", "Theme_Superheroes", "Theme_Circus"]:
        return "Adventure"
    elif theme in ['Fantasy', 'Mythology' , 'Theme_Vikings' , 'Theme_Witches' , 'Theme_Steampunk' , 'Theme_Ninjas' , 'Theme_King Arthur / The Knights of the Round Table / Camelot' , 'Theme_Samurai' , 'Theme_Kaiju' , 'Theme_Gladiators' , 'Theme_Alchemy']:
        return "Fantasy"
    elif theme in ['Space Exploration' , 'Science Fiction' , 'Theme_Post-Apocalyptic' , 'Theme_Time Travel' , 'Theme_Robots' , 'Theme_Mad Science / Mad Scientist', 'Theme_Cyberpunk']:
        return "Science Fiction"
    elif theme in ['Crime', 'Spies/Secret Agents', 'Mafia', 'Theme_Mystery / Cri', 'Theme_Villainy' , 'Theme_Jail / Prison (Modern)']:
        return "Crime / Underworld"
    elif theme in ['Fighting', 'Civil War', 'Modern Warfare', 'World War I', 'World War II', 'Pike and Shot', 'American Indian Wars' , 'Napoleonic', 'American Revolutionary War' , 'Vietnam War' , 'American Civil War' , 'Korean War' , 'Theme_Sieg' , 'Theme_Mech Warfar' , 'Theme_Animal Battles']:
        return "Warfare"
    elif theme in ['Environmental' , 'Farming', 'Animals' , 'Theme_Anthropomorphic Animals' , 'Theme_Gardening' , 'Theme_Flowers' , 'Theme_Natur' , 'Theme_Weather' , 'Theme_Evolution' , 'Theme_Fruit' ]:
        return "Nature"
    elif theme in ['Medical + Theme_Biology']:
        return "Medical"
    elif theme in ['Economic' , 'Industry / Manufacturing' , 'City Building' , 'Theme_Mining' , 'Theme_Construction' , 'Theme_City' , 'Theme_Oil / Gas / Petroleu']:
        return "Industrial"
    elif theme in ['Transportation' , 'Nautical' , 'Travel' , 'Trains' , 'Aviation / Flight' , 'Racing' , 'Theme_Submarines' , 'Theme_Amusement Parks / Theme Parks' ,'Theme_Airships / Blimps / Dirigibles / Zeppelins' , 'Theme_Firefighting']:
        return "Transportation"
    elif theme in ['Civilization', 'Age of Reason', 'Renaissance', 'American West', 'Medieval' , 'Ancient' , 'Post-Napoleonic' , 'Religious' , 'Arabian' , 'Prehistoric' , 'Theme_Alternate History' , 'Theme_Colonial' , 'Theme_Retro' ,'Theme_Deserts' , 'Theme_Tropical' , 'Theme_Native Americans / First Peoples' , 'Theme_Tropical Islands' , 'Theme_Safaris']:
        return "Historical Setting"
    elif theme in ['Movies / TV / Radio theme' , 'Music' , 'Theme_Art' , 'Theme_Archaeology / Paleontology' , 'Theme_Love / Romanc' , 'Theme_Boardgaming' , 'Theme_Movie Industry']:
        return "Cultural"
    elif theme in ['Horror' , 'Zombies' , 'Theme_Cthulhu Mythos' , 'Theme_Dreams / Nightmares' , 'Theme_Survival']:
        return "Horror"
    elif theme == 'Trivia':
        return "Trivia"
    elif theme in ['Sports' , 'Theme_Fantasy Sports']:
        return 'Sports'
    else:
        return 'Other'
    
    

# Exploring Users

We don't really want to share everyone's username, so we are going to use .cat.codes to encode them as numeric values. 

In [578]:
users2=users_df

In [579]:
# label encoding for username
users2['Username'] = users2['Username'].astype('category')
users2['Username'] = users2['Username'].cat.codes

We also only want to include the board games ratings we have in our database. (Sorry to all the players of [The Royal Game of Ur](https://en.wikipedia.org/wiki/Royal_Game_of_Ur)) 

In [580]:
boardgame_list = modern_boardgames_df['BGGId']
users_games_filtered = users2[users2['BGGId'].isin(boardgame_list)]
users_games_filtered

,BGGId,Rating,Username
0,213788,8.0,127828
1,213788,8.0,254518
2,213788,8.0,130672
3,213788,8.0,155121
4,213788,8.0,94049
...,...,...,...
18942177,165522,4.0,263072
18942178,165522,3.0,241524
18942179,165522,1.0,12931
18942213,193488,1.0,19655


Then we also want to focus on users with more than 5 reviews. 

In [582]:
# filter for users with more than x ratings
min_user_ratings = 5

rating_counter = users_games_filtered['Username'].value_counts()
user_mask = users_games_filtered['Username'].isin(rating_counter[rating_counter < min_user_ratings].index)

users_games_filtered.drop(index=users_games_filtered[user_mask].index, inplace=True)
users_games_filtered

/var/folders/4f/cr2zm2px70q9hrgj23zmw7_r0000gn/T/ipykernel_1287/677645296.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,BGGId,Rating,Username
0,213788,8.0,127828
1,213788,8.0,254518
2,213788,8.0,130672
3,213788,8.0,155121
4,213788,8.0,94049
...,...,...,...
18942177,165522,4.0,263072
18942178,165522,3.0,241524
18942179,165522,1.0,12931
18942213,193488,1.0,19655


That's.... still a lot of users. We'll take a random sample of 15,000, and collect all of their reviews. 

In [586]:
random_users = users_games_filtered['Username'].value_counts().sample(n=15000, random_state=42)
user_list = list(random_users.index)

In [587]:
sample_users = users_games_filtered[users_games_filtered['Username'].isin(user_list)]
sample_users

,BGGId,Rating,Username
20,213788,7.5,62528
26,213788,7.0,234851
48,213788,7.0,226898
53,213788,7.0,35217
62,213788,6.0,84277
...,...,...,...
18942113,165522,6.0,194929
18942120,165522,6.0,240379
18942128,165522,6.0,219617
18942172,165522,4.0,235242


In [588]:
sample_users.describe()

,BGGId,Rating,Username
count,865021.000000,865021.000000,865021.000000
mean,106509.059430,7.116095,140176.528929
std,91545.328866,1.511701,79425.400447
min,1.000000,0.001000,7.000000
25%,15062.000000,6.000000,71126.000000
50%,98918.000000,7.000000,142771.000000
75%,177736.000000,8.000000,210208.000000
max,345584.000000,10.000000,272144.000000


# Exporting Dataframes

In [589]:
modern_boardgames_df.to_csv("data/modern_games.csv")

In [590]:
sample_users.to_csv("data/users_encoded.csv")